# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [26]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [2]:
# Import the WeatherPy_database.csv file. 
cdf = pd.read_csv("WeatherPy_Database.csv")
# Display sample data
cdf.head()

,City_ID,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Vila Franca do Campo,PT,37.7167,-25.4333,59.41,79,74,10.92,broken clouds
1,1,Mahébourg,MU,-20.4081,57.7000,79.25,83,40,2.30,light rain
2,2,Punta Arenas,CL,-53.1500,-70.9167,68.11,40,0,28.77,clear sky
3,3,Atuona,PF,-9.8000,-139.0333,78.69,79,93,22.01,overcast clouds
4,4,Clyde River,CA,70.4692,-68.5914,-27.17,75,20,10.36,few clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [3]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = input("What is the minimum temperature you would like for your vacation? ")
max_temp = input("What is the maximum temperature you would like for your vacation? ")
print("Minimum Temperature: "+min_temp)
print("Maximum Temperature: "+max_temp)

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90
Minimum Temperature: 70
Maximum Temperature: 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
min_temp = pd.to_numeric(min_temp)
max_temp = pd.to_numeric(max_temp)
pcdf = cdf.loc[(cdf["Max Temp"].round().astype(int) >= min_temp) & (cdf["Max Temp"].round().astype(int) <= max_temp)]

# Display sample data
pcdf

,City_ID,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Mahébourg,MU,-20.4081,57.7000,79.25,83,40,2.30,light rain
3,3,Atuona,PF,-9.8000,-139.0333,78.69,79,93,22.01,overcast clouds
5,5,Itarema,BR,-2.9248,-39.9167,81.88,73,100,12.64,overcast clouds
10,10,Emerald,AU,-23.5333,148.1667,80.35,55,100,10.92,light rain
11,11,Port Alfred,ZA,-33.5906,26.8910,70.18,92,2,1.99,clear sky
...,...,...,...,...,...,...,...,...,...,...
685,685,Tumpat,MY,6.1978,102.1710,75.36,92,96,5.35,overcast clouds
686,686,Jalingo,NG,8.8833,11.3667,83.52,16,0,5.23,clear sky
689,689,Samarai,PG,-10.6167,150.6667,81.50,79,99,6.91,heavy intensity rain
690,690,Nakhon Nayok,TH,14.1667,101.1000,79.21,63,68,2.62,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [5]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
ctcities = pcdf.dropna(axis=0)
# Display sample data
ctcities.head()

,City_ID,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Mahébourg,MU,-20.4081,57.7000,79.25,83,40,2.30,light rain
3,3,Atuona,PF,-9.8000,-139.0333,78.69,79,93,22.01,overcast clouds
5,5,Itarema,BR,-2.9248,-39.9167,81.88,73,100,12.64,overcast clouds
10,10,Emerald,AU,-23.5333,148.1667,80.35,55,100,10.92,light rain
11,11,Port Alfred,ZA,-33.5906,26.8910,70.18,92,2,1.99,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = ctcities[["City", "Country", "Max Temp", "Weather Description", "Latitude", "Longitude"]].copy()
hotel_df.head()

,City,Country,Max Temp,Weather Description,Latitude,Longitude
1,Mahébourg,MU,79.25,light rain,-20.4081,57.7000
3,Atuona,PF,78.69,overcast clouds,-9.8000,-139.0333
5,Itarema,BR,81.88,overcast clouds,-2.9248,-39.9167
10,Emerald,AU,80.35,light rain,-23.5333,148.1667
11,Port Alfred,ZA,70.18,clear sky,-33.5906,26.8910


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [15]:
# Create a new column "Hotel Name".
hotel_df['Hotel Name'] = pd.Series(dtype='str')
# Display sample data
hotel_df

,City,Country,Max Temp,Weather Description,Latitude,Longitude,Hotel Name
1,Mahébourg,MU,79.25,light rain,-20.4081,57.7000,NaN
3,Atuona,PF,78.69,overcast clouds,-9.8000,-139.0333,NaN
5,Itarema,BR,81.88,overcast clouds,-2.9248,-39.9167,NaN
10,Emerald,AU,80.35,light rain,-23.5333,148.1667,NaN
11,Port Alfred,ZA,70.18,clear sky,-33.5906,26.8910,NaN
...,...,...,...,...,...,...,...
685,Tumpat,MY,75.36,overcast clouds,6.1978,102.1710,NaN
686,Jalingo,NG,83.52,clear sky,8.8833,11.3667,NaN
689,Samarai,PG,81.50,heavy intensity rain,-10.6167,150.6667,NaN
690,Nakhon Nayok,TH,79.21,broken clouds,14.1667,101.1000,NaN


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [16]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

In [20]:
latitude = -20.4081
longitude = 57.7000
    
# Add the current city's latitude and longitude to the params dictionary
params["filter"] = f"circle:{longitude},{latitude},{radius}"
params["bias"] = f"proximity:{longitude},{latitude}"
    
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v2/places?"

# Make request and retrieve the JSON data by using the params dictionary
name_address = requests.get(base_url, params=params)
    
# Convert the API response to JSON format
name_address = name_address.json()
name_address['features'][0]['properties']['name']


'Grand Bel Air'

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [21]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places?"

    # Make request and retrieve the JSON data by using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Mahébourg - nearest hotel: Grand Bel Air
Atuona - nearest hotel: Pearl Resort
Itarema - nearest hotel: Pousada Oásis
Emerald - nearest hotel: Emerald Central Hotel
Port Alfred - nearest hotel: No hotel found
Rikitea - nearest hotel: Chez Bianca & Benoit
Carnarvon - nearest hotel: No hotel found
Hithadhoo - nearest hotel: Pebbles Inn
Avarua - nearest hotel: Paradise Inn
Port Elizabeth - nearest hotel: Waterford Hotel
Monrovia - nearest hotel: Bella Casa Hotel
Akropong - nearest hotel: No hotel found
Baldwin - nearest hotel: No hotel found
São Filipe - nearest hotel: Bela Vista
Christchurch - nearest hotel: Ibis Hotel Christchurch
Meulaboh - nearest hotel: Hotel Meuligou
Vaini - nearest hotel: Keleti International Resort
Guane - nearest hotel: No hotel found
Cam Ranh - nearest hotel: Khách sạn Thái Bình
Mercedes - nearest hotel: Gran Hotel Mercedes
Kapaa - nearest hotel: Pono Kai Resort
Necochea - nearest hotel: Cabañas Río Quequén
Puerto Ayora - nearest hotel: Host

Kismayo - nearest hotel: Kismayo Hotel
Byron Bay - nearest hotel: Lord Byron Resort
Hambantota - nearest hotel: Oasis Aurweda Resort Hambathota
San Juan - nearest hotel: Hotel San Francisco
Karratha - nearest hotel: Karratha International Hotel
Iralaya - nearest hotel: No hotel found
Souillac - nearest hotel: Hotel SkylineInn
Prado - nearest hotel: Pousada Recanto do Prado
El Alto - nearest hotel: No hotel found
Sibolga - nearest hotel: Hotel Prima Indah
Berbérati - nearest hotel: Hotel Mont Fleury
Sulangan - nearest hotel: Marion's Inn
Guatire - nearest hotel: Grand Hotel
Malé - nearest hotel: Champa Central Hotel
Barão de Melgaço - nearest hotel: No hotel found
Quatre Cocos - nearest hotel: Horisun
Luganville - nearest hotel: Beachfront Resort
Padang - nearest hotel: losmen Surya
Kwang Binh - nearest hotel: Nhà hàng Amor Palace
Nhulunbuy - nearest hotel: No hotel found
Bom Jardim - nearest hotel: Hotel Breno
Nelson Bay - nearest hotel: No hotel found
High Rock - nearest hotel: No hot

,City,Country,Max Temp,Weather Description,Latitude,Longitude,Hotel Name
1,Mahébourg,MU,79.25,light rain,-20.4081,57.7000,Grand Bel Air
3,Atuona,PF,78.69,overcast clouds,-9.8000,-139.0333,Pearl Resort
5,Itarema,BR,81.88,overcast clouds,-2.9248,-39.9167,Pousada Oásis
10,Emerald,AU,80.35,light rain,-23.5333,148.1667,Emerald Central Hotel
11,Port Alfred,ZA,70.18,clear sky,-33.5906,26.8910,No hotel found
...,...,...,...,...,...,...,...
685,Tumpat,MY,75.36,overcast clouds,6.1978,102.1710,No hotel found
686,Jalingo,NG,83.52,clear sky,8.8833,11.3667,No hotel found
689,Samarai,PG,81.50,heavy intensity rain,-10.6167,150.6667,No hotel found
690,Nakhon Nayok,TH,79.21,broken clouds,14.1667,101.1000,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [24]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[(hotel_df['Hotel Name'] == 'No hotel found')].index)
# Display sample data
clean_hotel_df

,City,Country,Max Temp,Weather Description,Latitude,Longitude,Hotel Name
1,Mahébourg,MU,79.25,light rain,-20.4081,57.7000,Grand Bel Air
3,Atuona,PF,78.69,overcast clouds,-9.8000,-139.0333,Pearl Resort
5,Itarema,BR,81.88,overcast clouds,-2.9248,-39.9167,Pousada Oásis
10,Emerald,AU,80.35,light rain,-23.5333,148.1667,Emerald Central Hotel
18,Rikitea,PF,79.02,broken clouds,-23.1203,-134.9692,Chez Bianca & Benoit
...,...,...,...,...,...,...,...
669,Makakilo City,US,79.45,clear sky,21.3469,-158.0858,Embassy Suites by Hilton Oahu Kapolei
670,Ilhabela,BR,83.44,few clouds,-23.7781,-45.3581,Vila Kebaya
672,Salinópolis,BR,75.18,overcast clouds,-0.6136,-47.3561,Hotel Salinópolis
673,Koné,NC,76.59,overcast clouds,-21.0595,164.8658,Pacifik Appartels


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [25]:
# Create the output File (CSV)
fn = "WeatherPy_vacation.csv"
# Export the City_Data into a csv
clean_hotel_df.to_csv(fn, sep=',', index=False)

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [34]:
# Configure the map plot
vacmap = clean_hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    xaxis = None,
    yaxis = None,
    attr_labels = False,
    size = "Max Temp",
    hover = True,
    hover_cols = ["City", "Country", "Weather Description"]
)

# Display the map
vacmap

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Max Temp,City,Country,Weather Description)